<h1>Data Loading, Storage, and File Formats</h1>

Input and output typically fall into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

<h2>Reading and Writing Data in Text Format</h2>

Pandas features a number of functions for reading tabular data as a DataFrame object. This <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html">link</a> summarizes some of them; `pandas.read_csv` is one of the most frequently used in this section.

We will take an overview of the mechanics of these functions, which are meant to convert text data into a DataFrame. The optional arguments for these functions may fall into a few categories:

<b>Indexing</b><br />
Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, arguments you provide, or not at all.

<b>Type inference and data conversion</b><br />
Includes the user-defined value conversions and custom list of missing value markers.

<b>Date and time parsing</b><br />
Includes a combining capability, including combining date and time information spread over multiple columns into a single column in the result.

<b>Iterating</b><br />
Support for iterating over chunks of very large files.

<b>Unclean data issues</b><br />
Includes skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Because of how messy data in the real world can be, some of the data loading functions (especially `pandas.read_csv`) have accumulated a long list of optional arguments over time. It's normal to feel overwhelmed by the number of different parameters (`pandas.read_csv` has around 50). The online pandas documentation has many examples about how each of these works, so if you're struggling to read a particular file, there might be a similar enough example to help you find the right parameters.

Some of these functions perform type `inference`, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string. Other data formats, like HDF5, ORC, and Parquet, have the data type information embedded in the format.

Since this is comma-delimited, we can then use `pandas.read_csv` to read it into a DataFrame. To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/ex2.csv", header=None)
df

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
df = pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the `message` column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named `"message"` using the `index_col` argument:

In [4]:
names = ["a", "b", "c", "d", "message"]

pd.read_csv("../data/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


If you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:

In [5]:
parsed = pd.read_csv("../data/csv_mindex.csv", index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. While you could do some munging by hand, the fields here are separated by a variable amount of whitespace. In these cases, you can pass a regular expression as a delimiter for `pandas.read_csv`. This can be expressed by the regular expression `\s+`, so we have then:

In [6]:
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, `pandas.read_csv` infers that the first column should be the DataFrame’s index in this special case.
The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, second, and fourth rows of a file with skiprows:

In [7]:
pd.read_csv("../data/ex4.csv", skiprows=[0, 1, 3])

,,,,# just wanted to make things more difficult for you
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file reading process. Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`:

In [8]:
result = pd.read_csv("../data/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [9]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The `na_values` option accepts a sequence of strings to add to the default list of strings recognized as missing:

In [10]:
result = pd.read_csv("../data/ex5.csv", na_values=["NULL"])

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


`pandas.read_csv` has a list of many default `NA` value representations, but these defaults can be disabled with the `keep_default_na` option:

In [11]:
result2 = pd.read_csv("../data/ex5.csv", keep_default_na=False)

result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [12]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [60]:
result3 = pd.read_csv("../data/ex5.csv", keep_default_na=False, na_values=["NA"])

result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [61]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


<b>Note:</b> See pandas.read_csv function arguments in this <a href="https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html">link</a>.

<h2>Reading Text Files in Pieces</h2>

When processing very large files or figuring out the right set of arguments to correctly process a large file, you may want to read only a small piece of a file or iterate through smaller chunks of the file.

Before we look at a large file, we make the pandas display settings more compact:

In [15]:
pd.options.display.max_rows = 10

In [16]:
result = pd.read_csv("../data/ex6.csv")

result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


The elipsis marks `...` indicate that rows in the middle of the DataFrame have been omitted.

If you want to read only a small number of rows (avoiding reading the entire file), specify that with `nrows`:

In [63]:
pd_df = pd.read_csv("../data/ex6.csv", nrows=5)

In [64]:
pd_df["key"].value_counts()

L    1
B    1
G    1
R    1
Q    1
Name: key, dtype: int64

In [65]:
type(pd_df["key"].value_counts())

pandas.core.series.Series

To read a file in pieces, specify a `chunksize` as a number of rows:

In [18]:
chunker = pd.read_csv("../data/ex6.csv", chunksize=1000)

chunker

The `TextFileReader` object returned by `pandas.read_csv` allows you to iterate over the parts of the file according to the `chunksize`. For example, we can iterate over ex6.csv, aggregating the value counts in the "key" column, like so:

In [19]:
# Because `tot` is a series object the `add`` function
# add the value count based on the key from each chunk
tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [20]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

<h2>Writing Data to Text Format</h2>

Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:

In [21]:
data = pd.read_csv("../data/ex5.csv")

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s `to_csv` method, we can write the data out to a comma-separated file:

In [22]:
data.to_csv("../data/out.csv")

Other delimiters can be used, of course (writing to `sys.stdout` so it prints the text result to the console rather than a file):

In [23]:
import sys

data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:

In [24]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of these can be disabled:

In [25]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [26]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


<b>Note:</b> See pandas.DataFrame.to_csv function arguments in this <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html">link</a>.

<h2>Working with Other Delimited Formats</h2>

It's possible to load most forms of tabular data from disk using functions like `pandas.read_csv`. In some cases, however, some manual processing may be necessary. It’s not uncommon to receive a file with one or more malformed lines that trip up `pandas.read_csv`. To illustrate the basic tools, consider a small CSV file:

For any file with a single-character delimiter, you can use Python’s built-in `csv` module. To use it, pass any open file or file-like object to `csv.reader`:

In [27]:
import csv

f = open("../data/ex7.csv")

reader = csv.reader(f)

Iterating through the reader like a file yields lists of values with any quote characters removed:

In [28]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [29]:
f.close()

From there, it’s up to you to do the wrangling necessary to put the data in the form that you need. Let's take this step by step. First, we read the file into a list of lines:

In [30]:
with open("../data/ex7.csv") as f:
    lines = list(csv.reader(f))

Then we split the lines into the header line and the data lines:

In [66]:
header, values = lines[0], lines[1:]
header

['a', 'b', 'c']

In [32]:
values

[['1', '2', '3'], ['1', '2', '3']]

In [33]:
print(*values)

['1', '2', '3'] ['1', '2', '3']


Then we can create a dictionary of data columns using a dictionary comprehension and the expression `zip(*values)` (beware that this will use a lot of memory on large files), which transposes rows to columns:

In [34]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

<h2>JSON Data</h2>

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:

In [35]:
obj = """
        {"name": "Wes",
        "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
        "pet": null,
        "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
                    {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
        }
      """

JSON is very nearly valid Python code with the exception of its null value. The basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. I’ll use json here, as it is built into the Python standard library. To convert a JSON string to Python form, use json.loads:

In [36]:
import json

result = json.loads(obj)

result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

`json.dumps`, on the other hand, converts a Python object back to JSON:

In [37]:
asjson = json.dumps(result)

asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dictionaries (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields:

In [38]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])

siblings

,name,age
0,Scott,34
1,Katie,42


The `pandas.read_json` can automatically convert JSON datasets in specific arrangements into a Series or DataFrame. For example:

The default options for `pandas.read_json` assume that each object in the JSON array is a row in the table:

In [39]:
data = pd.read_json("../data/example.json")

data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


If you need to export data from pandas to JSON, one way is to use the to_json methods on Series and DataFrame:

In [40]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [41]:
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

<h2>XML and HTML: Web Scraping</h2>

Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.

pandas has a built-in function, pandas.read_html, which uses all of these libraries to automatically parse tables out of HTML files as DataFrame objects. To show how this works, I downloaded an HTML file (used in the pandas documentation) from the US FDIC showing bank failures. First, you must install some additional libraries used by read_html:

conda install lxml beautifulsoup4 html5lib

The `pandas.read_html` function has a number of options, but by default it searches for and attempts to parse all tabular data contained within `<table>` tags. The result is a list of DataFrame objects:

In [42]:
tables = pd.read_html("../data/fdic_failed_bank_list.html")
len(tables)

1

In [43]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


Because failures has many columns, pandas inserts a line break character `\`.

As you will learn in later chapters, from here we could proceed to do some data cleaning and analysis, like computing the number of bank failures by year:

In [44]:
close_timestamps = pd.to_datetime(failures["Closing Date"])

close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

<h3>Parsing XML with lxml.objectify</h3>

XML is another common structured data format supporting hierarchical, nested data with metadata. The book you are currently reading was actually created from a series of large XML documents.

Earlier, I showed the pandas.read_html function, which uses either lxml or Beautiful Soup under the hood to parse data from HTML. XML and HTML are structurally similar, but XML is more general. Here, I will show an example of how to use lxml to parse data from a more general XML format.

For many years, the New York Metropolitan Transportation Authority (MTA) published a number of data series about its bus and train services in XML format. Here we’ll look at the performance data, which is contained in a set of XML files. Each train or bus service has a different file (like Performance_MNR.xml for the Metro-North Railroad) containing monthly data as a series of XML records that look like this:

Using `lxml.objectify`, we parse the file and get a reference to the root node of the XML file with `getroot`:

In [45]:
from lxml import objectify

path = "../datasets/mta_perf/Performance_MNR.xml"

with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

`root.INDICATOR` returns a generator yielding each `<INDICATOR>` XML element. For each record, we can populate a dictionary of tag names (like `YTD_ACTUAL`) to data values (excluding a few tags) by running the following code:

In [46]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

Lastly, convert this list of dictionaries into a DataFrame:

In [47]:
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


Pandas's `pandas.read_xml` function turns this process into a one-line expression:

In [48]:
perf2 = pd.read_xml(path)

perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


<h2>Interacting with Web APIs</h2>

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one method that I recommend is the requests package, which can be installed with pip or conda:

conda install requests

To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

In [49]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"

resp = requests.get(url)

resp.raise_for_status()

resp

<Response [200]>

It's a good practice to always call `raise_for_status` after using `requests.get` to check for HTTP errors.

The response object’s `json` method will return a Python object containing the parsed JSON data as a dictionary or list (depending on what JSON is returned):

In [50]:
data = resp.json()

data[0]["title"]

' REGR: to_datetime with non-ISO format, float, and nan fails on main'

Since the results retrieved are based on real-time data, what you see when you run this code will almost definitely be different.

Each element in `data` is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass `data` directly to `pandas.DataFrame` and extract fields of interest:

In [51]:
issues = pd.DataFrame(data, columns=["number", "title", "labels", "state"])

issues

,number,title,labels,state
0,50238,"REGR: to_datetime with non-ISO format, float,...","[{'id': 211840, 'node_id': 'MDU6TGFiZWwyMTE4ND...",open
1,50237,"REGR: to_datetime with non-ISO format, float, ...","[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,50236,CLN: Remove Index._is_numeric_dtype,[],open
3,50235,DEPR: deprecate Index.is_float,[],open
4,50234,BUG: pd.Series.isin returns inconsistent resul...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,50196,DEPR: deprecate Index.is_interval,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open
26,50195,API: ensure IntervalIndex.left/right are 64bit...,"[{'id': 150096370, 'node_id': 'MDU6TGFiZWwxNTA...",open
27,50193,Index order fix,[],open
28,50192,BUG/TST Added support for empty dictionary whe...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open


<h2>Interacting with Databases</h2>

In a business setting, a lot of data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application.

pandas has some functions to simplify loading the results of a SQL query into a DataFrame. As an example, I’ll create a SQLite3 database using Python’s built-in `sqlite3` driver:

In [52]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("../data/mydata.sqlite")
con.execute(query)
con.commit()

Then, insert a few rows of data:

In [53]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

Most Python SQL drivers return a list of tuples when selecting data from a table:

In [54]:
cursor = con.execute("SELECT * FROM test")

rows = cursor.fetchall()

rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s `description` attribute. Note that for SQLite3, the cursor `description` only provides column names (the other fields, which are part of Python's Database API specification, are None), but for some other database drivers, more column information is provided:

In [55]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [56]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


This is quite a bit of munging that you’d rather not repeat each time you query the database. The SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. pandas has a read_sql function that enables you to read data easily from a general SQLAlchemy connection. You can install SQLAlchemy with conda like so:

`conda install sqlalchemy`

Now, we'll connect to the same SQLite database with SQLAlchemy and read data from the table created before:

In [58]:
import sqlalchemy as sqla

db = sqla.create_engine("sqlite:///../data/mydata.sqlite")

pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
